# Validate login by time, service and user

In [1]:
!pip install -r ../requirements.txt

  Cloning https://github.com/KindXiaoming/pykan.git (to revision v0.1.2) to /tmp/pip-req-build-7kke5dhx
  Running command git clone --filter=blob:none --quiet https://github.com/KindXiaoming/pykan.git /tmp/pip-req-build-7kke5dhx
  Running command git checkout -q bdf298a93a72ccf9a341ff95a8a2b92e243e50c4
  Resolved https://github.com/KindXiaoming/pykan.git to commit bdf298a93a72ccf9a341ff95a8a2b92e243e50c4
  Preparing metadata (setup.py) ... done
  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Using cached imageio-2.34.2-py3-none-any.whl (313 kB)
  Using cached imageio_ffmpeg-0.5.1-py3-none-manylinux2010_x86_64.whl (26.9 MB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


tt

Lode the test data

In [2]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
df = pd.read_csv('login_data/login_data.csv')
df.head()

,geography,service,user,weekday
0,Kristiansand,brukeranmeldelser,Kari,Sunday
1,Trondheim,meldinger,Kari,Sunday
2,Stavanger,innstillinger,Nils,Sunday
3,Bergen,innstillinger,Nils,Sunday
4,Stavanger,brukeranmeldelser,Per,Sunday


In [3]:

#df['user'].unique()
import ipywidgets as widgets
from IPython.display import display

# Hent en liste av unike brukere
unique_users = df['user'].unique()

# Lag en dropdown-liste med unike brukere
user_dropdown = widgets.Dropdown(options=unique_users)
# Vis dropdown-listen
display(user_dropdown)  # Viser brukeren 'per'

Dropdown(options=('Kari', 'Nils', 'Per', 'Anne', 'Ola'), value='Kari')

In [7]:
unique_services = df['service'].unique()
service_dropdown = widgets.Dropdown(options=unique_services)
display(service_dropdown)

Dropdown(options=('brukeranmeldelser', 'meldinger', 'innstillinger', 'profil'), value='brukeranmeldelser')

## User by service and weekday

In [8]:
!pip install seaborn

import seaborn as sns

In [9]:
# selectedUser = 'Ola'
selectedUser = user_dropdown.value
selectedService = service_dropdown.value
# Filtrer dataene for brukeren 'per' og tjenesten 'brukeranmeldelser'
filtered_df = df[(df['user'] == selectedUser) & (df['service'] == selectedService)]
# Få ukedagen som et tall
filtered_df['date'] = pd.to_datetime(filtered_df['date'])
filtered_df['weekday'] = filtered_df['date'].dt.dayofweek

# Grupper etter 'weekday' og tell antall pålogginger
weekday_counts = filtered_df['weekday'].value_counts().sort_index()
weekday_counts



KeyError: 'date'

In [10]:
# Lag en barplot for å visualisere antall pålogginger per ukedag
plt.figure(figsize=(10, 6))
sns.barplot(x=weekday_counts.index, y=weekday_counts.values, color='b')
plt.xlabel('Ukedag')
plt.ylabel('Antall pålogginger')
plt.title(f'Antall pålogginger per ukedag for brukeren "{selectedUser}" på tjenesten "{selectedService}"')
plt.show()

NameError: name 'weekday_counts' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:

# selectedUser = 'Ola'
# Filtrer dataene for brukeren 'per' og tjenesten 'brukeranmeldelser'
# filtered_df = df[(df['user'] == selectedUser) & (df['service'] == 'brukeranmeldelser')]

# Få ukedagen som et tall
# filtered_df['date'] = pd.to_datetime(filtered_df['date'])
# filtered_df['weekday'] = filtered_df['date'].dt.dayofweek

# Grupper etter 'weekday' og 'geography' og tell antall pålogginger
grouped_df = filtered_df.groupby(['weekday', 'geography']).size().reset_index(name='counts')

# Lag en gruppe barplot for å visualisere antall pålogginger per ukedag for hver geografisk region
plt.figure(figsize=(10, 6))
sns.barplot(x='weekday', y='counts', hue='geography', data=grouped_df, palette='viridis')
plt.xlabel('Ukedag')
plt.ylabel('Antall pålogginger')
plt.title(f'Antall pålogginger per ukedag for brukeren "{selectedUser}" på tjenesten "{selectedService}"')
plt.show()

# KAN

In [ ]:

from kan import *

from sklearn.model_selection import train_test_split
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

# KANimport matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Konvertere tid fra minutter siden midnatt til et tidspunkt i løpet av dagen
df['hour'] = df['time'] // 60

# Konverter 'date' til datetime
df['date'] = pd.to_datetime(df['date'])

# Få ukedagen som et tall
df['weekday'] = df['date'].dt.dayofweek

# Konvertere kategoriske data til numeriske verdier
df['weekday'] = df['weekday'].astype('category').cat.codes
df['geography'] = df['geography'].astype('category').cat.codes
df['service'] = df['service'].astype('category').cat.codes
df['user'] = df['user'].astype('category').cat.codes

# Velge relevante kolonner for KAN
data_for_kan = df[['weekday', 'geography', 'service', 'user']]
data_for_kan.head()

In [ ]:
# Skriv ut unike verdier fra 'weekday'-kolonnen
unique_values = df['weekday'].unique()
print(unique_values)

In [ ]:
# Konvertere til numpy arrays
X = data_for_kan.values
y = np.zeros(X.shape[0])  # Placeholder for målvariabel, kan brukes for unsupervised læring

# Konvertere til PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Splitte datasettet i trenings- og testsett
train_data, test_data, train_target, test_target = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Opprett data loaders (valgfritt, hvis du vil batch og shuffle data)
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data, train_target), batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data, test_target), batch_size=1, shuffle=False)

train_inputs = torch.empty(0, 4, device=device)
train_labels = torch.empty(0, dtype=torch.long, device=device)
test_inputs = torch.empty(0, 4, device=device)
test_labels = torch.empty(0, dtype=torch.long, device=device)

for data, labels in train_loader:
    train_inputs = torch.cat((train_inputs, data.to(device)), dim=0)
    train_labels = torch.cat((train_labels, labels.to(device)), dim=0)

for data, labels in test_loader:
    test_inputs = torch.cat((test_inputs, data.to(device)), dim=0)
    test_labels = torch.cat((test_labels, labels.to(device)), dim=0)

dataset = {
    'train_input': train_inputs,
    'test_input': test_inputs,
    'train_label': train_labels,
    'test_label': test_labels
}


In [ ]:
image_folder = 'video_img'
# Initialisere KAN med nødvendige parametere
model = KAN(width=[4, 5, 2], grid=5, k=3, seed=0, device=device)
model(dataset['train_input'])
model.plot(beta=100, scale=1, in_vars=['day', 'geo', 'service', 'user'], out_vars=['Ok', 'Fail'])

In [ ]:
 # Definere trenings- og testfunksjoner for nøyaktighet
def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).float())

# Trene modellen
results = model.train(dataset, opt="Adam", device=device, metrics=(train_acc, test_acc),
                      loss_fn=torch.nn.CrossEntropyLoss(), steps=100, lamb=0.01, lamb_entropy=10., save_fig=True, img_folder=image_folder)


print(f"Train accuracy: {results['train_acc'][-1]}, Test accuracy: {results['test_acc'][-1]}")


In [ ]:
model.plot(scale=1, in_vars=['SL', 'SW', 'PL', 'PW'], out_vars=['Set', 'Ver', 'Vir'])